Extraction and quantification of market sentiment from textual data such as news headlines or social media posts related to stock tickers. It fetches recent text data using AlphaVantage Global News API, preprocesses and cleans the text for analysis, and then applies the VADER sentiment analyser to assign sentiment scores (positive, negaitve, neutral and compound) to each piece of text. 
Scores are then aggregated over chosen time intervals to create a time-aligned sentiment dataset that can be merged with market price data for further modeling and visualisation. 

In [25]:
# import libraries 
import requests 
import pandas as pd 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime, timedelta

In [26]:
def fetch_alphavantage_news_df(api_url: str) -> pd.DataFrame: 
    """
    Fetches news data from Alphavantage Global News API URL and returns
    a cleaned pandas DataFrame with parsed dates.
    
    Parameters:
        api_url (str): Fully constructed API URL with key and parameters.
    
    Returns:
        pd.DataFrame: DataFrame with news articles.
    """
    response = requests.get(api_url)
    data = response.json()

    # Extract the "feed" list from the JSON response 
    news_list = data.get("feed", [])
    
    # Convert list of news dictionaries into a DataFrame
    df = pd.DataFrame(news_list)

    # Parse the published dates into datetime objects 
    df["time_published"] = pd.to_datetime(df["time_published"], errors = "coerce")

    return df

In [27]:
url = "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&date= &tickers=AAPL&apikey=1D1C3X346D6ATHPG"

In [28]:
# Dynamically create the url in order to specify the ticker, date, and apikey
def build_alphavantage_news_url(ticker, date, apikey):
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&date={date}&tickers={ticker}&apikey={apikey}"
    return url

news_url = build_alphavantage_news_url("AAPL", "2025-07-25","1D1C3X346D6ATHPG")

In [29]:
news_df = fetch_alphavantage_news_df(news_url)

In [30]:
# Visualise the dataframe 
print(news_df.columns.tolist())
print(news_df.head())

['title', 'url', 'time_published', 'authors', 'summary', 'banner_image', 'source', 'category_within_source', 'source_domain', 'topics', 'overall_sentiment_score', 'overall_sentiment_label', 'ticker_sentiment']
                                               title  \
0  Here Are My Top "Magnificent Seven" Stocks to ...   
1  After Sitting on the Sidelines For 14 Months, ...   
2  Apple's New Siri Voice-Control Feature Could R...   
3  Consumer Tech News  ( August 4-August 8 ) : Ea...   
4                          Is Apple Stock a Buy Now?   

                                                 url      time_published  \
0  https://www.fool.com/investing/2025/08/10/here... 2025-08-10 23:30:00   
1  https://www.fool.com/investing/2025/08/10/afte... 2025-08-10 22:23:00   
2  https://www.benzinga.com/markets/tech/25/08/47... 2025-08-10 19:43:01   
3  https://www.benzinga.com/markets/equities/25/0... 2025-08-10 12:31:08   
4  https://www.fool.com/investing/2025/08/10/is-a... 2025-08-10 09:30:00 

In [31]:
# Apply VADER sentiment analysis on the news headlines using the "title" and "summary" columns

sia = SentimentIntensityAnalyzer()
def get_sentiment_scores(text): 
    if isinstance(text, str):
        return sia.polarity_scores(text)
    else:
        return {'neg': None, 'neu': None, 'pos': None, 'compound': None}


In [32]:
# Create sentiment score columns 
news_df[["neg", "neu", "pos", "compound"]] = news_df["title"].apply(get_sentiment_scores).apply(pd.Series)

In [33]:
news_df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,neg,neu,pos,compound
0,"Here Are My Top ""Magnificent Seven"" Stocks to ...",https://www.fool.com/investing/2025/08/10/here...,2025-08-10 23:30:00,[Keithen Drury],"The ""Magnificent Seven"" stocks are still some ...",https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.327447,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1060...",0.000,0.584,0.416,0.6908
1,"After Sitting on the Sidelines For 14 Months, ...",https://www.fool.com/investing/2025/08/10/afte...,2025-08-10 22:23:00,[Adam Levy],Buffett bought shares of this stock for 23 str...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.265172,Somewhat-Bullish,"[{'ticker': 'AAPL', 'relevance_score': '0.0455...",0.000,0.857,0.143,0.4588
2,Apple's New Siri Voice-Control Feature Could R...,https://www.benzinga.com/markets/tech/25/08/47...,2025-08-10 19:43:01,[Bibhu Pattnaik],Apple's Siri is set to become your hands-free ...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.1...",0.351850,Bullish,"[{'ticker': 'GOOG', 'relevance_score': '0.0982...",0.000,1.000,0.000,0.0000
3,Consumer Tech News ( August 4-August 8 ) : Ea...,https://www.benzinga.com/markets/equities/25/0...,2025-08-10 12:31:08,[Nabaparna Bhattacharya],Ceasefire feelers and new U.S. tariffs on gold...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,General,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.036231,Neutral,"[{'ticker': 'APP', 'relevance_score': '0.07457...",0.000,1.000,0.000,0.0000
4,Is Apple Stock a Buy Now?,https://www.fool.com/investing/2025/08/10/is-a...,2025-08-10 09:30:00,[Keithen Drury],Apple has finally returned to double-digit gro...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.168318,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.2604...",0.000,1.000,0.000,0.0000
5,Apple's $600 Billion U.S. Investment Could Res...,https://www.fool.com/investing/2025/08/10/appl...,2025-08-10 07:45:00,[Patrick Sanders],Investors are hoping that Wednesday's announce...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.257684,Somewhat-Bullish,"[{'ticker': 'AMAT', 'relevance_score': '0.0508...",0.000,1.000,0.000,0.0000
6,Apple's AI Momentum Is Building -- Here's What...,https://www.fool.com/investing/2025/08/09/appl...,2025-08-09 17:00:00,[Stefon Walters],Apple plans to ramp up investments to catch up...,https://media.ycharts.com/charts/5f2f4d9e75818...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.242728,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.0622...",0.000,1.000,0.000,0.0000
7,"Microsoft, Apple, Amazon, and Meta Just Gave N...",https://www.fool.com/investing/2025/08/09/micr...,2025-08-09 13:05:00,[Jennifer Saibil],There's a lot of room for optimism as Nvidia i...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.208394,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1708...",0.000,0.608,0.392,0.7003
8,"Bulls And Bears: AMD, Apple, Eli Lilly - And W...",https://www.benzinga.com/markets/market-summar...,2025-08-09 12:19:59,[Benzinga Senior Editor],Benzinga examined the prospects for many inves...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.122102,Neutral,"[{'ticker': 'LLY', 'relevance_score': '0.14751...",0.000,0.949,0.051,0.2500
9,AAPL NEWS: Did

In [34]:
news_df.isnull().sum()

title                      0
url                        0
time_published             0
authors                    0
summary                    0
banner_image               0
source                     0
category_within_source     0
source_domain              0
topics                     0
overall_sentiment_score    0
overall_sentiment_label    0
ticker_sentiment           0
neg                        0
neu                        0
pos                        0
compound                   0
dtype: int64

In [40]:
### Exploration of Sentiment Data 

## Structural inspection 
# Check the first few rows 
print(news_df.head())

# Check the column names 
print("\nColumn names: ")
print(news_df.columns.tolist())

# Check data types and missing values 
print("\nDataFrame info:")
print(news_df.info())

# Check for missing values 
print("\nMissing values per column:")
print(news_df.isnull().sum())

# Quick statistics for numerical columns: 
print("\nSummary statistics:")
print(news_df.describe())

                                               title  \
0  Here Are My Top "Magnificent Seven" Stocks to ...   
1  After Sitting on the Sidelines For 14 Months, ...   
2  Apple's New Siri Voice-Control Feature Could R...   
3  Consumer Tech News  ( August 4-August 8 ) : Ea...   
4                          Is Apple Stock a Buy Now?   

                                                 url      time_published  \
0  https://www.fool.com/investing/2025/08/10/here... 2025-08-10 23:30:00   
1  https://www.fool.com/investing/2025/08/10/afte... 2025-08-10 22:23:00   
2  https://www.benzinga.com/markets/tech/25/08/47... 2025-08-10 19:43:01   
3  https://www.benzinga.com/markets/equities/25/0... 2025-08-10 12:31:08   
4  https://www.fool.com/investing/2025/08/10/is-a... 2025-08-10 09:30:00   

                    authors  \
0           [Keithen Drury]   
1               [Adam Levy]   
2          [Bibhu Pattnaik]   
3  [Nabaparna Bhattacharya]   
4           [Keithen Drury]   

                   

All the columns have the expected names and types 
There are 0 missing values 
The time_published is a datetime object which is to be expected